In [2]:
import pandas as pd
import re 
df = pd.read_csv('dados/teste.csv')
metade = len(df)/2
df_teste = df[:int(metade)]
df_treino = df[int(metade):]

In [3]:

def limpa_texto(df):
    pontuacao = [',', '.', ';', ':', '!', '?', '-', '(', ')', '[', ']', '{', '}', '"', '/', '\\', '|', '_', '+', '=', '*', '&', '@', '#', '$', '%', '^', '<','>','`','~']
    lista = []
    for palavras in df:
        lista.append(palavras)
    string = ' '.join(lista)
    if '<br /><br />' in string:
        string = string.replace('<br /><br />', ' ')
    for pontos in pontuacao:
        if pontos in string:
            string = string.replace(f'{pontos}','')
    lista = re.split(r'\s+|\W+', string.lower())
    lista = [palavra for palavra in lista if palavra]
    return lista

lista_total = limpa_texto(df_teste['review'])


filtro_positivo = df_teste['sentiment'] == 'positive'
df_positivo = df_teste[filtro_positivo]
lista_positivo = limpa_texto(df_positivo['review'])


filtro_negativo = df_teste['sentiment'] == 'negative'
df_negativo = df_teste[filtro_negativo]
lista_negativo = limpa_texto(df_negativo['review'])

In [4]:

dic_contagem = {}
for palavras in lista_positivo:
    if palavras not in dic_contagem:
        dic_contagem[palavras] = 1
    else:
        dic_contagem[palavras] += 1

dic_probabilidades_positivo = {}
for palavra, contagem in dic_contagem.items():
    dic_probabilidades_positivo[palavra] = int(contagem)/len(lista_total)

dic_probabilidades_barrada_positivo = {}
for palavra, probabilidade in dic_probabilidades_positivo.items():
    dic_probabilidades_barrada_positivo[palavra] = 1-probabilidade

for palavras in lista_negativo:
    if palavras not in dic_probabilidades_barrada_positivo:
        dic_probabilidades_barrada_positivo[palavras] = 1

In [5]:
dic_contagem = {}
for palavras in lista_negativo:
    if palavras not in dic_contagem:
        dic_contagem[palavras] = 1
    else:
        dic_contagem[palavras] += 1

dic_probabilidades_negativo = {}
for palavra, contagem in dic_contagem.items():
    dic_probabilidades_negativo[palavra] = int(contagem)/len(lista_total)

dic_probabilidades_barrada_negativo = {}
for palavra, probabilidade in dic_probabilidades_negativo.items():
    dic_probabilidades_barrada_negativo[palavra] = 1-probabilidade

for palavras in lista_positivo:
    if palavras not in dic_probabilidades_barrada_negativo:
        dic_probabilidades_barrada_negativo[palavras] = 1

In [12]:
import math 
p_positiva = 1
p_positiva_barrada=1
p_negativa = 1
p_negativa_barrada=1
indicie_acerto=0
zero_positivo=False
zero_negativo=False
dic={}
for i in range(len(df_teste)):
    lista=df_teste.iloc[i]['review'].split(' ') 
    string = ' '.join(lista)
    if '<br /><br />' in string:
        string = string.replace('<br /><br />', ' ')

    pontuacao = [',', '.', ';', ':', '!', '?', '-', '(', ')', '[', ']', '{', '}', '"', '/', '\\', '|', '_', '+', '=', '*', '&', '@', '#', '$', '%', '^', '<','>','`','~']
    for pontos in pontuacao:
        if pontos in string:
            string = string.replace(f'{pontos}','')

    lista_arrumada = re.split(r'\s+|\W+', string.lower())
    lista_arrumada = [palavra for palavra in lista_arrumada if palavra]
    for palavras in lista_arrumada:
        if palavras in dic_probabilidades_positivo:
            p_positiva += math.log(dic_probabilidades_positivo[palavras])
        if palavras in dic_probabilidades_negativo:
            p_negativa += math.log(dic_probabilidades_negativo[palavras])

        if palavras not in dic_probabilidades_positivo:
            p_positiva_barrada += math.log(dic_probabilidades_barrada_positivo[palavras])
            zero_positivo=True 
        if palavras not in dic_probabilidades_negativo:
            p_negativa_barrada += math.log(dic_probabilidades_barrada_negativo[palavras])
            zero_negativo=True 
        
    print(p_positiva,p_negativa)
    if zero_positivo==True:
        dic[i]='negative'
    elif zero_negativo==True:
        dic[i]='positive'
    else:
        if p_positiva>p_negativa:
            dic[i]='positive'
        else:
            dic[i]='negative'
for i in dic:
    if dic[i]==df_teste.iloc[i]['sentiment']:
        indicie_acerto+=1
print((indicie_acerto*100)/len(df_teste))
print(dic)

-2495.802268619856 -2446.6573028283847
-3723.4850117437486 -3627.2678725448163
-4982.3778999625765 -4877.5690472040815
-5984.039806314546 -5871.082100146459
-7722.8772401953765 -7578.2928372408805
-8645.417778045045 -8498.3711105108
-9826.856910510964 -9523.383300221218
-11087.173948980057 -10802.93549672126
-11981.482558856733 -11710.393225302501
-12218.769773783984 -11948.624736082224
-12923.579067639044 -12646.117427949237
-14220.428362964558 -13947.333317229919
-17273.53107482505 -17192.75973442137
-18073.520633016396 -18023.158200409303
-18436.720047011906 -18391.847129921254
-19507.690752775008 -19462.011148321933
-20514.943799749846 -20457.529963106408
-22150.632802446817 -22126.131820077604
-23128.24481588814 -23016.62044415082
-24143.090559254833 -24040.149922273267
-26696.632901934652 -26455.565261563388
-27965.82421646886 -27719.452095058157
-28465.30170142977 -28203.971775711867
-30834.43621902333 -30615.939969117808
-31846.647120369187 -31663.753814086824
-32724.4865437860